<a href="https://colab.research.google.com/github/giyeongyoon/lettuce_growth_monitoring/blob/main/lettuce_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load dataset

In [1]:
# %%capture
# !pip install ultralytics
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="Ql1dOvGZp8vqTAnCTntx")
# project = rf.workspace("kiyong-kang-qhti3").project("plants-2l8n0")
# dataset = project.version(8).download("yolov8")

Import libraries

In [4]:
import os
import shutil
import copy
import yaml
from ultralytics import YOLO
import glob
import cv2
import matplotlib.pyplot as plt
import random
from PIL import Image
%matplotlib inline

Create segmentation directory

In [3]:
if not os.path.exists('./Plants-seg/train/images'):
    os.makedirs('./Plants-seg/train/images')

if not os.path.exists('./Plants-seg/train/labels'):
    os.makedirs('./Plants-seg/train/labels')

if not os.path.exists('./Plants-seg/valid/images'):
    os.makedirs('./Plants-seg/valid/images')

if not os.path.exists('./Plants-seg/valid/labels'):
    os.makedirs('./Plants-seg/valid/labels')

In [4]:
t_labels = os.listdir('./Plants-8/train/labels')
v_labels = os.listdir('./Plants-8/valid/labels')

train_src = './Plants-8/train'
val_src = './Plants-8/valid'

train_dir = './Plants-seg/train'
val_dir = './Plants-seg/valid'

Filter segmentation files

In [5]:
def filter_seg_files(src, labels):
    seg_files = []

    for label_name in labels:
        filename = label_name[:-4]
        label_dir = src + '/labels/' + filename + '.txt'
        with open(label_dir, 'r') as f:
            objs = f.readlines()

        for obj in objs:
            ann = obj.split()
            if len(ann) - 1 > 4:
                seg_files.append(filename)

    seg_files = list(set(seg_files))

    return seg_files

In [6]:
seg_t_files = filter_seg_files(train_src, t_labels)
seg_v_files = filter_seg_files(val_src, v_labels)

In [7]:
print('Number of train data: ', len(t_labels))
print('Number of train tip burn data: ', len(seg_t_files))
print('Number of valid data: ', len(v_labels))
print('Number of valid valid data: ', len(seg_v_files))

Number of train data:  1517
Number of train tip burn data:  59
Number of valid data:  133
Number of valid valid data:  0


Move tip burn images/labels for segmentation

In [8]:
def move_files(src, dir, files):
    for filename in files:
        shutil.move(src + '/images/' + filename + '.jpg', dir + '/images/' + filename + '.jpg')
        shutil.move(src + '/labels/' + filename + '.txt', dir + '/labels/' + filename + '.txt')

In [9]:
move_files(train_src, train_dir, seg_t_files)
move_files(val_src, val_dir, seg_v_files)

In [10]:
print('Number of train data: ', len(os.listdir(train_src + '/images')), '/', len(os.listdir(train_src + '/labels')))
print('Number of valid data: ', len(os.listdir(val_src + '/images')), '/', len(os.listdir(val_src + '/labels')))
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of train data:  1458 / 1458
Number of valid data:  133 / 133
Number of tipburn train images/labels:  59 / 59
Number of tipburn valid images/labels:  0 / 0


In [11]:
x = os.listdir(train_dir + '/images')
x_val = random.sample(x, 6)
x_val = [i[:-4] for i in x_val]

move_files(train_dir, val_dir, x_val)

# for filename in x_val:
#     shutil.move(train_dir + '/images/' + filename + 'jpg', val_dir + '/images/' + filename + 'jpg')
#     shutil.move(train_dir + '/labels/' + filename + 'txt', val_dir + '/labels/' + filename + 'txt')

In [12]:
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of tipburn train images/labels:  53 / 53
Number of tipburn valid images/labels:  6 / 6


In [13]:
def modify_seg_files(src, labels):
    seg_files = []

    for label_name in labels:
        filename = label_name[:-4]
        label_dir = src + '/labels/' + filename + '.txt'
        img_dir = src + '/images/' + filename + '.jpg'
        with open(label_dir, 'r') as f:
            objs = f.readlines()

        n_obj_tipburn = 0
        objs_tipburn = []
        objs_others = []

        for obj in objs:
            if obj[0] == '0':
                n_obj_tipburn += 1
                objs_tipburn.append(obj)
            else:
                objs_others.append(obj)

        if n_obj_tipburn == len(objs):
            continue
        elif n_obj_tipburn < len(objs) and n_obj_tipburn > 0:
            with open(label_dir, 'w') as f2:
                f2.writelines(objs_tipburn)
        elif n_obj_tipburn == 0:
            os.remove(img_dir)
            os.remove(label_dir)

In [14]:
t_seg_labels = os.listdir('./Plants-seg/train/labels')
v_seg_labels = os.listdir('./Plants-seg/valid/labels')

modify_seg_files(train_dir, t_seg_labels)
modify_seg_files(val_dir, v_seg_labels)

In [15]:
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of tipburn train images/labels:  50 / 50
Number of tipburn valid images/labels:  6 / 6
